In [35]:
import psycopg
import requests
from label_studio_sdk.client import LabelStudio
from psycopg.rows import dict_row
from tqdm.auto import tqdm

from fishsense_data_processing_spider.config import PG_CONN_STR, settings

In [37]:
dive_checksums_to_promote = [
    '68849aca8957a057d667a15cca98522e',
    '2b8502abfe2e0820158da616839edd7b',
    '54fbd79fbc58460d747d4aa69c716afa'
]
low_priority_laser_project = 43
high_priority_laser_project = 42
low_priority_headtail_project = 45
high_priority_headtail_project = 44

In [21]:
with open('.secrets/n_hui_api_key', 'r', encoding='utf-8') as handle:
    fs_api_key = handle.read().strip()

In [22]:
with open(settings.postgres.password_file, 'r', encoding='utf-8') as handle:
    pg_pasword = handle.read().strip()

In [23]:
client = LabelStudio(
    base_url=f'https://{settings.label_studio.host}',
    api_key=settings['label_studio']['api_key']
)

In [25]:
frames = []
with requests.Session() as session:
    session.headers.update({
        'api_key': fs_api_key
    })
    for dive in dive_checksums_to_promote:
        req = session.get(
            url=f'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/metadata/dive/{dive}'
        )
        doc = req.json()
        frames.extend(doc['frames'])

In [38]:
low_priority_laser_tasks = [task
                      for task in client.tasks.list(
                        project=low_priority_laser_project
                      )
                      ]
low_priority_headtail_tasks = [task
                            for task in client.tasks.list(
                                project=low_priority_headtail_project
                            )
                            ]

In [39]:
relevant_laser_tasks = {task.data['img'].split('/')[-1]:task
                  for task in low_priority_laser_tasks
                  if task.data['img'].split('/')[-1] in frames}
relevant_headtail_tasks = {task.data['img'].split('/')[-1]: task
                           for task in low_priority_headtail_tasks
                        if task.data['img'].split('/')[-1] in frames}

In [40]:
new_laser_tasks = []
new_headtail_tasks = []
with psycopg.connect(PG_CONN_STR) as con, con.cursor() as cur:
    for cksum, task in tqdm(relevant_laser_tasks.items()):
        if len(task.annotations) > 0:
            continue
        client.tasks.delete(task.id)
        new_task = client.tasks.create(
            data=task.data,
            project=high_priority_laser_project
        )
        cur.execute(
            query=('UPDATE laser_labels SET task_id = %(id)s WHERE cksum = %(cksum)s;'),
            params={
                'id': new_task.id,
                'cksum': cksum
            }
        )
        new_laser_tasks.append(new_task)
    for cksum, task in tqdm(relevant_headtail_tasks.items()):
        if len(task.annotations) > 0:
            continue
        client.tasks.delete(task.id)
        new_task = client.tasks.create(
            data=task.data,
            project=high_priority_headtail_project
        )
        cur.execute(
            query=('UPDATE headtail_labels SET task_id = %(id)s WHERE cksum = %(cksum)s;'),
            params={
                'id': new_task.id,
                'cksum': cksum
            }
        )
        new_headtail_tasks.append(new_task)

0it [00:00, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]